In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# ! cat "/content/drive/My Drive/CS221/transformers-master/src/transformers/tokenization_auto.py"
# ! cat "/content/drive/My Drive/CS221/transformers-master/src/transformers/modeling_auto.py"
import sys
sys.path.append('/content/drive/My Drive/CS221/transformers-master/src')

In [ ]:
! pip install transformers
! pip install sacremoses
! pip install bert-extractive-summarizer

     |████████████████████████████████| 1.3MB 5.2MB/s 
     |████████████████████████████████| 890kB 31.7MB/s 
     |████████████████████████████████| 2.9MB 44.1MB/s 
     |████████████████████████████████| 1.1MB 41.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=64f21d62f8b6dce0b73dc3c6ae1d585ead2e2a1b99b2bb664f92913c4f9e0476
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/CS221/Data/bbc_data_clean_summary.csv')

In [ ]:
dataset.head()

,Unnamed: 0,File,Text,Summary
0,0,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,the dollar has hit its highest level against t...
1,1,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,yukos owner menatep group says it will ask ros...
2,2,006.txt,Japan narrowly escapes recession\n\nJapan's ec...,on an annual basis the data suggests annual gr...
3,3,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,pernod has reduced the debt it took on to fund...
4,4,007.txt,Jobs growth still slow in the US\n\nThe US cre...,the job gains mean that president bush can cel...


In [ ]:
for i in range(len(dataset)):
  print(len(dataset["Summary"].get(i)), end = ", ")

916, 672, 431, 597, 765, 856, 1158, 659, 547, 965, 1680, 494, 1362, 1254, 628, 232, 672, 818, 563, 760, 888, 557, 851, 1034, 489, 546, 673, 510, 1496, 1064, 661, 698, 591, 999, 702, 439, 335, 580, 724, 1005, 629, 928, 517, 879, 848, 678, 1248, 846, 925, 688, 620, 620, 1511, 424, 515, 715, 811, 888, 582, 700, 798, 797, 872, 1251, 1209, 519, 1056, 866, 574, 623, 1084, 611, 521, 762, 572, 573, 1300, 673, 716, 429, 516, 400, 505, 593, 471, 412, 916, 928, 1003, 641, 1124, 562, 1046, 942, 877, 524, 911, 691, 673, 1362, 472, 743, 1009, 903, 947, 285, 905, 425, 534, 860, 1054, 1285, 1250, 474, 525, 811, 688, 517, 449, 739, 792, 599, 655, 539, 486, 985, 490, 941, 1045, 753, 773, 1342, 873, 1298, 581, 980, 685, 1201, 1344, 1069, 495, 652, 1084, 295, 915, 664, 641, 727, 1007, 998, 854, 1545, 796, 607, 495, 723, 1003, 742, 990, 507, 903, 594, 576, 538, 580, 759, 2241, 1964, 662, 727, 724, 760, 424, 842, 648, 851, 610, 715, 1099, 539, 919, 1063, 689, 672, 1450, 1045, 1210, 729, 805, 957, 727, 1340,

Code

In [ ]:
from summarizer import TransformerSummarizer
import re

In [ ]:
def remove_special_characters_and_to_lowercase(s):
    # remove special characters except A-Z, a-z, 0-9, and space.
    s = re.sub(r'[^A-Za-z0-9 ]+', '', s)
    # to lowercase
    return s.lower()

In [ ]:
model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'h.12.attn.masked_bias', 'h.13.attn.masked_bias', 'h.14.attn.masked_bias', 'h.15.attn.masked_bias', 'h.16.attn.masked_bias', 'h.17.attn.masked_bias', 'h.18.attn.masked_bias', 'h.19.attn.masked_bias', 'h.20.attn.masked_bias', 'h.21.attn.masked_bias', 'h.22.attn.masked_bias', 'h.23.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def runModel(input: str):
  raw_output = ''.join(model(input, min_length=40, max_length=150))
  output = remove_special_characters_and_to_lowercase(raw_output)
  return ''.join(output)

Evaluation

In [ ]:
# Because the actual output (500~1000 words) is usually much longer than the
# predicted output, instead of how many n-grams are matched in the actual
# output, we compute the number of bigrams in the predicted output that also
# appear in the actual output.
#
# Params
# n: n-gram to compare.
def computeCustomizedRougeN(predicted_output: str, actual_output: str, n: int):
  predicted_words = predicted_output.split()
  actual_words = actual_output.split()
  assert n <= len(predicted_words) and n <= len(actual_words)
  predicted_ngrams = [''.join(predicted_words[i : i + n]) for i in range(len(predicted_words) - n + 1)]
  actual_ngrams = set([''.join(actual_words[i : i + n]) for i in range(len(actual_words) - n + 1)])
  num_matches = 0
  for predicted_ngram in predicted_ngrams:
    if predicted_ngram in actual_ngrams:
      num_matches += 1
  return num_matches / len(predicted_ngrams)

In [ ]:
computeCustomizedRougeN("I have apple", "I have an apple", 2)  # 0.5 since "I have" is a match, but "have apple" is not.

0.5

In [ ]:
dictionary = {'Text': [], 'Summary': [], 'Rouge2': [], 'Rouge3': [], 'Rouge4': []}

for i in range(dataset["Text"].size):
  print('=' * 10, "Round", i, '=' * 10)
  input = dataset["Text"].get(i)
  predicted_output = runModel(input)
  actual_output = dataset["Summary"].get(i)
  custom_rouge2_score = computeCustomizedRougeN(predicted_output, actual_output, 2)
  custom_rouge3_score = computeCustomizedRougeN(predicted_output, actual_output, 3)
  custom_rouge4_score = computeCustomizedRougeN(predicted_output, actual_output, 4)
  print("Input:", input)
  print("Predicted Output:", predicted_output)
  print("Actual Output:", actual_output)
  print("Customized Rouge2 score:", custom_rouge2_score)
  print("Customized Rouge3 score:", custom_rouge3_score)
  print("Customized Rouge4 score:", custom_rouge4_score)
  dictionary['Text'].append(input)
  dictionary['Summary'].append(predicted_output)
  dictionary['Rouge2'].append(custom_rouge2_score)
  dictionary['Rouge3'].append(custom_rouge3_score)
  dictionary['Rouge4'].append(custom_rouge4_score)

df = pd.DataFrame(dictionary)
output_csv = "GPT2_output.csv"
df.to_csv(output_csv)
print("CSV saved!")

Streaming output truncated to the last 5000 lines.
The international exclusion zone around georgewbush.com was spotted by net monitoring firm Netcraft which keeps an eye on traffic patterns across many different sites. Netcraft said that since the early hours of 25 October attempts to view the site through its monitoring stations in London, Amsterdam and Sydney failed.

By contrast Netcraft's four monitoring stations in the US managed to view the site with no problems. The site can still be seen using anonymous proxy services that are based in the US. Some web users in Canada also report that they can browse the site. The pattern of traffic to the website suggests that the blocking was not due to an attack by vandals or politically motivated hackers. Geographic blocking works because the numerical addresses that the net uses to organise itself are handed out on a regional basis. On 21 October, the George W Bush website began using the services of a company called Akamai to ensure that 